In [76]:
from math import ceil
import time, re, ast, sys, urllib, random, string, requests
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from itertools import cycle
from header_list import user_agent_list
from proxies_list import proxies_list_
from INITIAL_SCRAPPER_FUNCTIONS import *
from PARSER_FUNCTIONS import *
from LIST_DF_FUNCTIONS import *
from SEARCH_URL_GEN import *
from GET_SEARCH_URL import *
ua = user_agent_list
proxies = proxies_list_

In [13]:
customer_url = input("Input the Redfin URL of the Home: ")

Input the Redfin URL of the Home: 


In [14]:
# customer_df = each_property(customer_url, header, proxy)

In [34]:
customer_df = pd.DataFrame(columns=['address', 'city', 'state', 'zip', 'type',
                                    'price', 'sqft', 'num_bds', 'num_bths',
                                    'lot_sf', 'yr_blt', 'HOA', 'HOA_fee'])

In [77]:
def gen_url(customer_df):
    zip_code = int(customer_df['zip'].values)
    city = str(customer_df['city'].values)
    type_home = (customer_df['home_type'].values)
    price = float(customer_df['price'].values)
    num_bds = int(customer_df['num_bds'].values)
    num_bths = float(customer_df['num_bths'].values)
    sqft = int(customer_df['sqft'].values)
    yr_blt = int(customer_df['yr_blt'].values)
    lot_sqft = int(customer_df['lot_sf'].values)
    hoa = customer_df['HOA'].values
    hoa_fee = float(customer_df['HOA_fee'].values)

    url_part_one = 'https://www.redfin.com/zipcode/' + \
        str(zip_code) + '/filter/sort=lo-days/'

    url_part_two = search_url_part_two_gen(type_home)

    url_part_three = search_url_part_three_gen(price)

    url_part_four = search_url_part_four_gen(num_bds)

    url_part_five = search_url_part_five_gen(num_bths)

    url_part_six = search_url_part_six_gen(sqft)

    url_part_seven = search_url_part_seven_gen(yr_blt)

    url_part_eight = search_url_part_eight_gen(lot_sqft)

    url_part_nine = search_url_part_nine_gen(hoa)

    search_url = url_part_one + url_part_two + url_part_three + \
        url_part_four + url_part_five + url_part_six + \
        url_part_seven + url_part_eight + url_part_nine

    return search_url

In [78]:
active_zip_url_list = gen_active_url_list(zip_)
df = pd.DataFrame(columns=['full_address', 'home_link'])
active_zip_url_list, df, proxies = links_for_active_props(proxies, active_zip_url_list, df, ua)
# for url in df['home_link']:
#     active_df = each_property(url, header, proxy)
# return active_df[0:2]

NameError: name 'gen_active_url_list' is not defined

world , i'm a beginner 
